 # Running Log Analytics
 
 Created by: Miles Smith, Aubrey Roberts
 
 Jan. 3, 2024

In [1]:
import os.path
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import seaborn as sns

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

/Users/milessmith/anaconda3/envs/conda-env/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Import running log data using Google Workspace API

### Define User Inputs

In [2]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
SPREADSHEET_ID = "1VdRxTTKiQk_9wtxOaegy8d-PMaSG-ijkFDCBGHjY_nw"
SHEET_ID = "1403524267"
RANGE_NAME = "ACTIVITY LOG!A:AH"

### Set Credentials

In [3]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())


RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

### Import Data from Sheets

In [51]:
try:
    service = build("sheets", "v4", credentials=creds)
    
    # Call sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME)
        .execute()
        )
    
    values = result.get("values", [])

    
    if not values:
        print("No data found.")
    
    # Clean data and set data types
    for i in range(len(values)):
        # Ignore empty rows.
        if values[i][5] and i > 0:
            # Define the rows from activity log
            values[i][0] = int(values[i][0]) # Day number
            values[i][1] = bool(values[i][1] == 'TRUE') # EOW
            values[i][2] = bool(values[i][2] == 'TRUE') # EOM
            values[i][3] = bool(values[i][3] == 'TRUE') # EOY
            values[i][4] = str(values[i][4]) # Day
            values[i][5] = datetime.strptime(str(values[i][5]), '%m/%d/%Y').date() # Date
            values[i][6] = float(values[i][6]) # Distance
            try:
                values[i][7] = datetime.strptime(str(values[i][7]), '%H:%M:%S').time() # Duration
            except ValueError:
                values[i][7] = None
            try:
                values[i][8] = datetime.strptime(str(values[i][8]), '%H:%M:%S').time() # Pace
            except:
                values[i][8] = None
            values[i][9] = str(values[i][9]) # Activity Type
            try:
                values[i][10] = int(values[i][10]) # Elevation Gain
            except ValueError:
                values[i][10] = None
            try:
                values[i][11] = int(values[i][11]) # Heart Rate
            except ValueError:
                values[i][11] = None
            try:    
                values[i][12] = int(values[i][12]) # Cadence
            except ValueError:
                values[i][12] = None
            values[i][13] = str(values[i][13]) # 
            values[i][14] = str(values[i][14]) # 
            values[i][15] = bool(values[i][15] == 'TRUE') # Drills
            values[i][16] = bool(values[i][16] == 'TRUE') # Strides
            values[i][17] = bool(values[i][17] == 'TRUE') # Strength
            try:
                values[i][18] = float(values[i][18]) # Sleep Hours
            except ValueError:
                values[i][18] = None
            values[i][19] = str(values[i][19]) # Sleep Quality
            values[i][20] = str(values[i][20]) # Energy
            values[i][21] = bool(values[i][21] == 'TRUE') # Nap
            values[i][22] = float(values[i][22]) # WTD Easy
            values[i][23] = float(values[i][23]) # WTD Hard
            values[i][24] = float(values[i][24]) # WTD Long
            values[i][25] = float(values[i][25]) # WTD Bike
            values[i][26] = float(values[i][26]) # WTD Other
            values[i][27] = str(values[i][27]) # WTD XT Time
            values[i][28] = str(values[i][28]) # WTD Run Time
            values[i][29] = float(values[i][29]) # WTD
            values[i][30] = float(values[i][30]) # MTD
            values[i][31] = float(values[i][31]) # YTD
            values[i][32] = float(values[i][32]) # Total Distance
            try:
                values[i][33] = str(values[i][33]) # Equipment
            except IndexError:
                values[i].append(None) 
    
except HttpError as err:
    print(err)
    
run_df = pd.DataFrame(data=values[1:],columns=values[0])
run_df = run_df[run_df['Date'] != ''] # Remove empty rows from data frame.

# print(run_df['Energy'])

## Running Analytics

### Linear Regression

#### Future Steps:
- Adding effort and energy levels might be interesting and are tracked on the Google Sheet
- Import time, elevation, and location data from activities using Garmin or Strava API
- Import weather data from OpenWeatherMap API
- How to incorporate changes in fitness?
- What if non-linear?

In [ ]:
easy_df = run_df
type_dummy_df = pd.get_dummies(easy_df['Type'], prefix='Type')

energy_order = ['Barely functioning', 'Low', 'Normal', 'High']
feel_order = ['Awful', 'Bad', 'Normal', 'Good', 'Great']
effort_order = ['Easy', 'Moderate', 'Hard', 'Max Effort']

easy_df['energy_encoded'] = easy_df['Energy'].astype('category').cat.set_categories(energy_order).cat.codes.replace(-1, None)
easy_df['feel_encoded'] = easy_df['Feel'].astype('category').cat.set_categories(feel_order).cat.codes.replace(-1, None)
easy_df['effort_encoded'] = easy_df['Effort'].astype('category').cat.set_categories(effort_order).cat.codes.replace(-1, None)

easy_df = pd.concat([easy_df, type_dummy_df], axis=1)

X = easy_df[['Distance (mi.)', 'Duration (hh:mm:ss)', 'Pace (min/mile)', 'Climb (ft.)', 'Cadence', 'Sleep (hours)', 'energy_encoded', 'Type_Long Run', 'Type_Easy', 'feel_encoded', 'effort_encoded', 'HR (bpm)']]
X = X.dropna()
X['Duration (hh:mm:ss)'] = X['Duration (hh:mm:ss)'].apply(lambda x: x.hour * 60 + x.minute)
X['Pace (min/mile)'] = X['Pace (min/mile)'].apply(lambda x: x.hour * 60 + x.minute)
y = X['HR (bpm)']
X = X.drop('HR (bpm)', axis=1)

lin_reg = LinearRegression()

lin_reg.fit(X, y)
y_predict = lin_reg.predict(X)

lin_reg_fig, lin_reg_ax = plt.subplots(1)
lin_reg_ax.plot(y.values, y_predict, linewidth=0, marker='.')
lin_reg_ax.set_xlim(120, 180)
lin_reg_ax.set_ylim(120, 180)
lin_reg_ax.set_ylabel('Predicted Heart Rate (bpm)')
lin_reg_ax.set_xlabel('Measured Heart Rate (bpm)')

rmse = math.sqrt(mean_squared_error(y, y_predict))
r_squared = r2_score(y, y_predict)

X = np.array(X, dtype=float)
y = np.array(y, dtype=float)

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

model.summary()

### Heart Rate Analytics

In [ ]:
colormap = plt.get_cmap('Set2')
easy_df['pace_float'] = easy_df['Pace (min/mile)'].apply(lambda x: x.minute+x.second/60)

# Interesting variables: pace, day (box plot)
hr_ax = easy_df.plot(x='pace_float', y='HR (bpm)', kind='scatter', c=easy_df['effort_encoded'], cmap=colormap)
plt.colorbar(hr_ax.get_children()[0], label='Effort')
hr_ax.set_ylim(120, 200)
hr_ax.set_title('Average heart rate compared to average pace')
hr_ax.set_xlabel('Pace (min/mile)')
hr_ax.set_ylabel('Heart Rate (bpm)')

### Partial autocorrelation for seasonal trends
- How can we explain the variance in heart rate centered around 7:00/mile?
- partial autocorrelation function with sleep, pace
- narrow temperature (6:45 - 7:15 maybe)
- incorporate predictors such as time of day, elevation, temperature, time of year
- multi-variate autocorrelation function with relative weights for time series
- look into system identification 
- running research group
- follow up with joey licht

### Sleep Analytics

In [ ]:
easy_df = run_df
easy_df = easy_df[easy_df['Sleep (hours)'] != ''] # Remove empty rows from data frame.
easy_df = easy_df[easy_df['HR (bpm)'] != ''] # Remove empty rows from data frame.
easy_df =easy_df.dropna()
sns.boxplot(x='Sleep (hours)', y='HR (bpm)', data=easy_df)
plt.ylim(120, 170)

### Mileage Bar Chart

In [ ]:
week_fig, week_ax = plt.subplots(1)
wtd_df = run_df[run_df['EOW'] == True]

width=6
week_ax.bar(wtd_df['Date'], wtd_df['Easy'], width, label='Easy')
week_ax.bar(wtd_df['Date'], wtd_df['Hard'], width, label='Hard', bottom=np.array(wtd_df['Easy']))
week_ax.bar(wtd_df['Date'], wtd_df['Long'], width, label='Long', bottom=np.add(np.array(wtd_df['Easy']), np.array(wtd_df['Hard'])))
week_ax.legend()
week_ax.set_ylabel('Weekly Mileage (mi.)')
week_ax.set_xlabel('End of Week')
week_ax.set_title('Weekly Mileage')
week_fig.set_figwidth(25)
week_fig.set_figheight(10)

## Import data from Garmin Connect

In [6]:
from garminconnect import Garmin
import datetime

In [10]:
start_date = datetime.datetime(2015, 1, 1)
end_date = datetime.date.today()

# Initialize API
garmin_api = Garmin()
garmin_api.login()
activities = garmin_api.get_activities(0, 10000)

AssertionError: 

In [45]:
for activity in activities:
    print(activity)

for i in range(len(activities)):
    print(activities[i]['startTimeLocal'])
    # print(activities[i]['averageHR'])